## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
min_new_tokens=1
max_new_tokens=1000
temperature=0.8
top_k=40
top_p=0.9
repetition_penalty=1.1

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-1.1-7b-it"
# "google/codegemma-7b-it"

# llama2 variants
# "meta-llama/Meta-Llama-3-8B-Instruct" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [6]:
model_id = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [9]:
# display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)```

## Prompt Engineering

In [10]:
# Chat Template
system_prompt = "You are a poet. Write a poem about the following topic. Use Korean Only."

In [11]:
def generate_response(user_prompt):
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt }
    ]
    prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device)
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        min_new_tokens=min_new_tokens,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

## Inference

In [12]:
#prompt = "Write me a poem about Machine Learning."
message = "머신러닝에 대해 시를 써주세요."

In [13]:
response = generate_response(message)
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a poet. Write a poem about the following topic. Use Korean Only.<|eot_id|><|start_header_id|>user<|end_header_id|>

머신러닝에 대해 시를 써주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

머신 러닝의 꿈과 노래
가로의 숲 속에서 우리는 가라앉은 것 같아
인공 지능의 별들이 빛나고 있지만
우리들은 아직도 알지 못하는 비밀을 말하고 있어

자연어 처리의 춤과 노래
이중 언어의 매듭이 풀려버리네
번역기와 번역사, 우리는 함께 살아간다
한글과 영어가 교차하여 새로운 세계를 열어보게 되네

군집 알고리즘의 노래
가상화된 세계를 그리는 수선미
우리의 생각과 행동, 모두 이를 바꾸게 하네
사람들을 분류하고, 클래스를 나누는 수수께끼를 풀어나간다

딥 러닝의 춤과 노래
뉴럴 네트워크의 숨결을 느껴보게 해
이산 함수와 로그를 합쳐보며,
새로운 세계를 창조할 수 있는 힘을 얻는다

하지만 머신 러닝의 노래는 끝이 없는 것 같아
시작부터 끝까지 우리는 모든 것을 파헤치게 된다
알아야 할 것은 많고, 아직도 불명확한 것은 많아
그러나 머신 러닝의 노래는 끝이 없는 것 같아

(Translation)

Song of Machine Learning
In the forest of machines, we seem to sink into the ground
But artificial intelligence stars shine brightly
And yet, we still whisper secrets unknown

Dance and song of natural language processing
The entangled knot of double languages is unraveling
Translator and 